# Assignment #5: Probability and Bootstrapping
## Foundations of Machine Learning
## ` ! git clone https://www.github.com/DS3001/the_bootstrap`
## Do two.

**Q1.** A die is fair if every face is equally likely. A die has six sides if it has six faces labelled 1, 2, ... , 6.

1. Imagine rolling two dice, $d_1$ and $d_2$. Let $R_{min}$ be the lesser value of the two face values. What is the probability of getting a 1, 2, 3, 4, 5, or 6? Which values are more or less likely compared to the roll of a single six-sided die? What is the expected value of $R_{min}$? Plot the probability and cumulative distribution functions for $R_{min}$. Compute this by hand and simulate it using the law of large numbers.
2. Imagine rolling three dice, $d_1$, $d_2$, and $d_3$. Let $R_{med}$ be the middle of the three face values. So if you roll 2, 3 and 4, the middle value is 3, and if you roll 2, 4, 4, the middle value is 4, and so on. What is the probability of getting a 1, 2, 3, 4, 5, or 6? Which values are more or less likely compared to the roll of a single six-sided die? What is the expected value of $R_{med}$? Plot the probability and cumulative distribution functions for $R_{med}$. I recommend using simulations and the law of large numbers.
3. Imagine rolling a die. If you roll 1, 2, 3, 4, or 5, add that number to your total and stop; if you roll a six, add that number to your total and roll the die again. So you could roll, say, two sixes and then a four, and get a total of 16, or one three and get a total of 3, or twelve sixes and 1 and get 72, and so on. Write code to simulate this process, and determine its expected value using the law of large numbers. What is the probability of getting a total of 1, 2, 3, ... and so on, in your simulation? I recommend using simulations and the law of large numbers.  (Hint: The `while` loop might be useful in this case.)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

faces = np.arange(1, 7)

pdf = np.zeros(len(faces))
for die1 in faces:
    for die2 in faces:
        minimum = min(die1, die2)
        pdf[minimum - 1] += 1
pdf /= 36

cdf = np.cumsum(pdf)

num_simulations = 1000
rolls_die1 = np.random.choice(faces, size=num_simulations)
rolls_die2 = np.random.choice(faces, size=num_simulations)
min_values = np.minimum(rolls_die1, rolls_die2)

empirical_cdf = np.array([np.mean(min_values <= x) for x in faces])

plt.figure(figsize=(8, 6))
plt.step(faces, empirical_cdf, label='Empirical CDF', color='green', where='mid')
plt.plot(faces, cdf, label='Theoretical CDF', color='orange', marker='o')
plt.xlabel("Minimum Value (M)")
plt.ylabel("Cumulative Probability")
plt.title("Comparison of Empirical and Theoretical CDFs")
plt.legend()
plt.grid()
plt.show()

print("PDF (Theoretical):", pdf)
print("CDF (Theoretical):", cdf)
print("CDF (Empirical):", empirical_cdf)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

faces = np.array([1, 2, 3, 4, 5, 6])

N = 5000
rolls_die1 = np.random.choice(faces, size=N)
rolls_die2 = np.random.choice(faces, size=N)
rolls_die3 = np.random.choice(faces, size=N)

middle_values = [int(np.median([rolls_die1[i], rolls_die2[i], rolls_die3[i]])) for i in range(N)]

ecdf = np.array([(1 / N) * np.sum(np.array(middle_values) <= face) for face in faces])

single_cdf = np.array([(1 / N) * np.sum(rolls_die1 <= face) for face in faces])

expected_value = np.mean(middle_values)

plt.figure(figsize=(8, 6))
plt.plot(faces, ecdf, marker='o', linestyle='-', label='Median Value CDF', color='green')
plt.plot(faces, single_cdf, marker='x', linestyle='--', label='Single Die CDF', color='blue')
plt.xlabel("Value")
plt.ylabel("Cumulative Probability")
plt.title("CDF Comparison: Median vs Single Die Roll")
plt.legend()
plt.grid()
plt.show()

# Output results
print(f"Expected Value of the Median: {expected_value:.4f}")
print("Empirical CDF (Median):", ecdf)
print("Empirical CDF (Single Die):", single_cdf)


In [ ]:
faces = np.array([1, 2, 3, 4, 5, 6])

N = 50000
results = []
for _ in range(N):
    total = 0
    while True:
        roll = np.random.choice(faces)
        total += roll
        if roll < 6:
            results.append(total)
            break

results_df = pd.DataFrame(results, columns=['Total'])

print(results_df.describe())

plt.figure(figsize=(10, 5))
sns.histplot(results_df['Total'], bins=30, kde=True, color='blue')
plt.title('Histogram of Totals')
plt.xlabel('Total')
plt.ylabel('Frequency')
plt.grid()
plt.show()

plt.figure(figsize=(10, 5))
sns.ecdfplot(results_df['Total'], color='green')
plt.title('Empirical CDF of Totals')
plt.xlabel('Total')
plt.ylabel('Cumulative Probability')
plt.grid()
plt.show()

expected_value = results_df['Total'].mean()
print(f"Expected Value of Total: {expected_value:.4f}")


**Q2.** This question refers to the `mammogram.csv` data. It has two variables, `treatment` which takes the values `control` or `mammogram`, and `breast_cancer_death`, which takes the values `no` or `yes`. This is an experiment that followed 89,835 women for 25 years to see if mammograms were superior to more traditional breast cancer screenings in preventing breast cancer deaths.

1. Cross tabulate `treatment` and `breast_cancer_death`. What is the difference in 25-year survival rates between the control and mammogram groups?
2. Bootstrap the densities and distributions of survival rates for the two groups.
3. Construct a 99% confidence interval for the difference in outcomes bewteen the two groups. Does it include zero?
4. We're not doctors, these were just some intriguing data, and the information about the patients is extremely sparse. Why might these data over/understate the conclusions you've reached? What other data would you like to have to better understand or criticize your results?

In [ ]:
df = pd.read_csv('./data/mammogram.csv')

print(pd.crosstab(df['treatment'], df['breast_cancer_death']), '\n')
print(pd.crosstab(df['treatment'], df['breast_cancer_death'], margins=True, normalize=True), '\n')

control_survival_rate = 44425 / 44925
mammogram_survival_rate = 44405 / 44910
raw_treatment_effect = mammogram_survival_rate - control_survival_rate
print('Raw treatment effect: ', raw_treatment_effect)

df['survive'] = (df['breast_cancer_death'] == 'no').astype(int)

df_treat = df[df['treatment'] == 'mammogram']
df_control = df[df['treatment'] == 'control']

S = 5000
bootstrap_differences = []
for _ in range(S):
    treat_sample = df_treat.sample(n=df_treat.shape[0], replace=True)
    control_sample = df_control.sample(n=df_control.shape[0], replace=True)
    treatment_effect = treat_sample['survive'].mean() - control_sample['survive'].mean()
    bootstrap_differences.append(treatment_effect)

sns.kdeplot(bootstrap_differences, color='blue')
plt.title('Bootstrap Density of Treatment Effects')
plt.xlabel('Difference in Survival Rates')
plt.ylabel('Density')
plt.grid()
plt.show()

sns.ecdfplot(bootstrap_differences, color='green')
plt.title('Bootstrap ECDF of Treatment Effects')
plt.xlabel('Difference in Survival Rates')
plt.ylabel('Cumulative Probability')
plt.grid()
plt.show()

lower_bound = np.quantile(bootstrap_differences, 0.005)
upper_bound = np.quantile(bootstrap_differences, 0.995)
print(f"99% confidence interval: ({lower_bound:.5f}, {upper_bound:.5f})")



The 99% confidence interval includes zero, which suggests that getting a mammogram doesn’t lead to a statistically significant difference in 25-year survival rates. There are various questions that can be asked about how the data was collected and about the results. For example, are wealthier, healthier individuals were more likely to get mammograms, the treatment group might naturally have a lower prevalence of breast cancer, making mammograms seem less effective.

We’d also need to consider other risk factors for breast cancer. While mammograms might not reduce mortality rates at the population level, they could be highly effective for certain subpopulations or high-risk groups. Plus, this analysis only focuses on mortality. Mammograms might improve quality of life in other ways, such as catching breast cancer earlier and reducing the need for intensive treatments like chemotherapy, even if survival rates are similar.

# 4. Discussion
print("Potential biases and limitations:")
print("1. Lack of patient demographic information such as age, genetics, or prior health conditions may lead to confounding factors.")
print("2. The data do not account for differences in the quality or timing of mammograms or other treatments.")
print("3. Results could overstate conclusions if other mortality causes are not controlled for.")
print("Additional data needed:")
print("1. Detailed patient demographics and health histories.")
print("2. Information on the timing and type of screenings.")
print("3. Broader context about other causes of death and overall healthcare access.")


**Q3.** This question refers to the `diabetes_hw.csv` data. It contains two variables, `outcome` and `treatment`. Each is looking at whether an individual's diabetes was successfully treated (`outcome==success`) with `lifestyle` interventions like exercises and diets, a drug denoted by `met` (metformin), or a drug denoted by `rosi` (rosiglitazone), or not (`outcome==failure`). Treatment success means that the individual no longer needs to be treated with insulin, while failure means the patient still required insulin injections after treatment.

1. Cross tabulate `treatment` and `outcome`.
2. Compute the the proportion of successes for each treatment. Which treatment appears to be the most effective?
3. Bootstrap the density and distribution of the proportion of successes for each interventions. Create empirical CDF and kernel density plots that are grouped  by treatment type. Which treatment appears to be the most effective?
4. For each comparison (lifestyle versus met, met versus rosi, rosi versus lifestyle), bootstrap the distribution of the difference in outcomes. At the 90% level of confidence, which pairwise treatment comparisons are significantally different?
5. Which treatment appears to be the most effective overall?

**Q4.** The goal of the question is to incorporate features/covariates/predictors/explanatory variables into the kind of treatment effect comparisons done in the previous questions. This question refers to the `heart_hw.csv` data. It contains three variables:

  - `y`: Whether the individual survived, coded 0 for death and 1 for survival
  - `age`: Patient's age
  - `transplant`: `control` for not receiving a transplant and `treatment` for receiving a transplant

1. Compute (a) the proportion of people who survive in the control group who do not receive a transplant, and (b) the difference between the proportion of people who survive in the treatment group and the proportion of people who survive in the control group (the average treatment effect).
2. Regress `y` on `transplant` using a linear model. How does the constant/intercept of the regression and the coefficient on transplant compare to your answers from part 1? Explain carefully.
3. We'd like to include `age` in the regression, since it's reasonable to expect that older patients are less likely to survive an extensive surgery like a heart transplant. Regress `y` on transplant, age, and transplant $\times$ age. You can do this using a linear regression. How do the intercept and the coefficient on `transplanttreatment` change?
4. Build a more flexible model that allows for non-linear age effects and interactions between age and treatment. Estimate the model, and plot the predicted survival probability by age, hued for people who receive a heart transplant and those who don't. Describe what you see.
5. Imagine someone suggests using these kinds of models to select who receives organ transplants; perhaps the CDC or NIH starts using a scoring algorithm to decide who is contacted about a potential organ transplant. What are your concerns about how it is built and how it is deployed?